In [3]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import errorcode
import re
import unicodedata

def create_database(cursor, db_name):
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")

def connect_to_database(host, user, password, db_name):
    db = mysql.connector.connect(host=host, user=user, password=password)
    cursor = db.cursor()
    create_database(cursor, db_name)
    db.database = db_name
    return db, cursor

def detect_data_type(value):
    try:
        int(value.replace(',', ''))
        return "INT"
    except ValueError:
        try:
            float(value.replace(",", "."))
            return "FLOAT"
        except ValueError:
            return "VARCHAR(255)"

def normalize_column_name(column_name):
    normalized_name = re.sub(r'[^\w]+', '', column_name)
    return normalized_name.upper() if normalized_name else f"UNNAMED_COLUMN"

def create_table(cursor, data, table_name):
    columns = [f"`{normalize_column_name(key)}` {detect_data_type(val[0])}" for key, val in data.items()]
    columns_str = ', '.join(columns)
    sql_create = f"CREATE TABLE IF NOT EXISTS `{table_name}` (id INT AUTO_INCREMENT PRIMARY KEY, {columns_str})"
    cursor.execute(sql_create)

def insert_data(cursor, data, table_name):
    column_names = [normalize_column_name(key) for key in data.keys()]
    columns = ', '.join(f"`{name}`" for name in column_names)
    placeholders = ', '.join(['%s'] * len(data))
    sql_insert = f"INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})"
    cursor.execute(sql_insert, list(data.values()))

def scrape_table(url, cursor, db):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    if not table:
        print(f"No se encontró ninguna tabla en la URL {url}.")
        return

    headers = [th.get_text(strip=True) for th in table.find_all('th')]
    rows = table.find_all('tr')[1:]
    data_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == len(headers):
            row_data = {headers[i]: cells[i].get_text(strip=True) for i in range(len(cells))}
            data_list.append(row_data)

    if data_list:
        table_name = re.sub(r'\W+', '', url.split('/')[-1]) or "default_table"
        create_table(cursor, data_list[0], table_name)
        for data in data_list:
            insert_data(cursor, data, table_name)
        db.commit()
        print(f"Datos insertados correctamente para {url}.")
    else:
        print(f"No se encontraron datos en la tabla de {url}.")

def main():
    url = input("https://countryeconomy.com/business/motor-vehicle-in-use")
    db, cursor = connect_to_database("localhost", "root", "", "scraping_db")
    try:
        scrape_table(url, cursor, db)
    finally:
        if db and db.is_connected():
            cursor.close()
            db.close()

if __name__ == "__main__":
    main()

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión)